In [ ]:
df = pd.read_excel('Refined_dataset.xlsx') 
def preprocess_df(df):
    df['state']=[st.lower() for st in df['state']]
    df['categoryName']=[cat.lower() for cat in df['categoryName']]
    df['city']=[ct.lower() for ct in df['city']]
    return df

In [ ]:
class category_index:
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.cat_list=[st.lower() for st in df['categoryName'].unique()]

  def get_cat_index(self,cat):
    cat_df=self.dataframe[self.dataframe['categoryName']==cat]
    #print(cat)
    return list(cat_df.index.values)

  def make_index(self):
    for cat in self.cat_list:
      self.index_dict[cat.lower()]=self.get_cat_index(cat)
      
  def get_cat_list(self):
    return self.cat_list


In [ ]:
from google.colab import files
uploads=files.upload()

Saving state_acr.txt to state_acr.txt
Saving synonym_cat.txt to synonym_cat.txt


In [ ]:
cat_syn_obj=open('/content/synonym_cat.txt','r')#synonym creation code
cat_syn_text=cat_syn_obj.read()
cat_syn_dict=dict()
lines=cat_syn_text.split(";")
for ln in lines:
  ln_spl=ln.split(":")
  print(ln_spl)
  print(ln_spl[1].split(","))
  for wd in ln_spl[1].split(","):
    wd=wd.replace("'","")
    wd=wd.replace("\n","")
    wd=wd.strip()
    cat_syn_dict[str(wd).lower()]=str(ln_spl[0]).replace("\n","").lower()
  cat_syn_dict[str(ln_spl[0]).replace("\n","").lower()]=str(ln_spl[0]).replace("\n","").lower()
st_syn_obj=open('/content/state_acr.txt','r')
st_syn_text=st_syn_obj.read()
st_syn_dict=dict()
lines_st=st_syn_text.split(";")
for ln in lines_st:
  ln_spl=ln.split(":")
  wd1=ln_spl[1].replace('\n','')
  wd1=wd1.strip()
  st_syn_dict[str(wd1).lower()]=str(ln_spl[0]).replace("\n","").lower()
  st_syn_dict[str(ln_spl[0]).replace("\n","").lower()]=str(ln_spl[0]).replace("\n","").lower()

['Pilgrimage', "'Hindu Temple','Buddhist temple','Pagoda','Church','Catholic church','Jain temple','Cathedral','Shrine','Gurudwara','Mosque','Religious,destination','Bahá'í house of worship','Anglican church','Monastery','Meditation center','Place of worship','Pilgrimage place','Catholic cathedral','Basilica','Cemetery','Tomb','Spiritist center','Religious organization','Protestant church','Religious institution','Christian church','Baptist church','Presbyterian church'"]
["'Hindu Temple'", "'Buddhist temple'", "'Pagoda'", "'Church'", "'Catholic church'", "'Jain temple'", "'Cathedral'", "'Shrine'", "'Gurudwara'", "'Mosque'", "'Religious", "destination'", "'Bahá'í house of worship'", "'Anglican church'", "'Monastery'", "'Meditation center'", "'Place of worship'", "'Pilgrimage place'", "'Catholic cathedral'", "'Basilica'", "'Cemetery'", "'Tomb'", "'Spiritist center'", "'Religious organization'", "'Protestant church'", "'Religious institution'", "'Christian church'", "'Baptist church'", "

In [ ]:
print(cat_syn_dict)
print(st_syn_dict)

{'hindu temple': 'pilgrimage', 'buddhist temple': 'pilgrimage', 'pagoda': 'pilgrimage', 'church': 'pilgrimage', 'catholic church': 'pilgrimage', 'jain temple': 'pilgrimage', 'cathedral': 'pilgrimage', 'shrine': 'pilgrimage', 'gurudwara': 'pilgrimage', 'mosque': 'pilgrimage', 'religious': 'pilgrimage', 'destination': 'pilgrimage', 'baháí house of worship': 'pilgrimage', 'anglican church': 'pilgrimage', 'monastery': 'pilgrimage', 'meditation center': 'pilgrimage', 'place of worship': 'pilgrimage', 'pilgrimage place': 'pilgrimage', 'catholic cathedral': 'pilgrimage', 'basilica': 'pilgrimage', 'cemetery': 'pilgrimage', 'tomb': 'pilgrimage', 'spiritist center': 'pilgrimage', 'religious organization': 'pilgrimage', 'protestant church': 'pilgrimage', 'religious institution': 'pilgrimage', 'christian church': 'pilgrimage', 'baptist church': 'pilgrimage', 'presbyterian church': 'pilgrimage', 'pilgrimage': 'pilgrimage', 'beach': 'beach', 'public beach': 'beach', 'island': 'beach', 'beach pavilli

In [ ]:
import pickle as pkl   # synonym pickling
fobj=open("synonym_cat_dict.pkl",'wb')
pkl.dump(cat_syn_dict,fobj)
fobj.close()
fobj1=open("synonym_st_dict.pkl",'wb')
pkl.dump(st_syn_dict,fobj1)
fobj1.close()

files.download('synonym_cat_dict.pkl')     # downloading the pkl file for storing locally
files.download('synonym_st_dict.pkl')     

In [ ]:
class MainIndex:
  def __init__(self):
    self.cat_i=None
    self.st_i=None
    self.city_i=None
    self.cross_index=dict()
    self.cross_list=list()

  def get_indices(self):
    fobj1=open('cat_index.pkl','rb')
    self.cat_i=pkl.load(fobj1)
    fobj1.close()
    fobj2=open('state_index.pkl','rb')
    self.st_i=pkl.load(fobj2)
    fobj2.close()
    fobj3=open('city_index.pkl','rb')
    self.city_i=pkl.load(fobj3)
    fobj3.close()

  def merger(self,cat,st='',ct=''):
    if len(ct)>0:
      l1=self.cat_i.get_cat_index(cat)
      l2=self.ct_i.get_ct_index(ct)
      fin_index=set(l1).intersection(set(l2))
      return list(fin_index)
    elif len(st)>0:
      l1=self.cross_index[cat]
      l2=self.cross_index[st]
      fin_index=set(l1).intersection(set(l2))
      return list(fin_index)
    else:
      return []

  def create_cross_index(self):
    self.get_indices()
    for cat in self.cat_i.get_cat_list():
        self.cross_index[cat]=list()
        self.cross_index[cat].extend(self.cat_i.get_cat_index(cat))
    for st in self.st_i.get_state_list():
      self.cross_index[st]=list()
      self.cross_index[st].extend(self.st_i.get_st_index(st))
    for city in self.city_i.get_ct_list():
        if city in self.st_i.get_state_list():
              self.cross_index[city].extend(self.city_i.get_city_index(city))
        else:
              self.cross_index[city]=list()
              self.cross_index[city].extend(self.city_i.get_city_index(city))
    for cat in self.cat_i.get_cat_list():
      for st in self.st_i.get_state_list():
        tup=tuple([cat,st])
        self.cross_index[tup]=self.merger(cat,st)
    for cat in self.cat_i.get_cat_list():
      for ct in self.city_i.get_ct_list():
        tup=tuple([cat,ct])
        self.cross_index[tup]=self.merger(cat,ct)

  def index_lookup(self,cat='',pl=''):
    print("category",cat)
    print("place",pl)
    if len(cat)>0:
      if len(pl)>0:
        tup=tuple([cat,pl])
        return self.cross_index[tup]
      else:
        return self.cross_index[cat]
    elif len(pl)>0:
      return self.cross_index[pl]
    else:
      return []



In [ ]:
uploads=files.upload()            # upload cat_index, state_index, city_index
mi = MainIndex()
mi.create_cross_index()
fobj=open('cross_index.pkl','wb')
pkl.dump(mi,fobj)
fobj.close()
files.download('cross_index.pkl')     